In [1]:
!pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   - -------------------------------------- 11.8/390.3 MB 61.4 MB/s eta 0:00:07
   -- ------------------------------------- 26.7/390.3 MB 65.1 MB/s eta 0:00:06
   ---- ----------------------------------- 40.9/390.3 MB 65.0 MB/s eta 0:00:06
   ----- ---------------------------------- 51.9/

  You can safely remove it manually.
  You can safely remove it manually.


In [27]:
!pip install ipywidgets


   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 33.4 MB/s eta 0:00:00


Import the necessary modules

In [62]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from math import ceil
from collections import defaultdict
from tqdm import tqdm # progress bar


In [17]:

!pip install tqdm

Import deep learning frameworks

In [18]:
import tensorflow as tf

# image feature extraction
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# captions tokenization
from tensorflow.keras.preprocessing.text import Tokenizer

# padding sequences to a specific length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# models
from tensorflow.keras.models import Model, load_model

# layers
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, LayerNormalization, MultiHeadAttention, concatenate

# utils
from tensorflow.keras.utils import plot_model, to_categorical

# checking the score
from nltk.translate.bleu_score import corpus_bleu

In [22]:
# input and output directories
INPUT_DIR = r'C:\Users\gauta\OneDrive\Desktop\Image-Captioner\Image-captioning\Data'
OUTPUT_DIR = r'C:\Users\gauta\OneDrive\Desktop\Image-Captioner\Image-captioning\Results'

if not os.path.exists(INPUT_DIR):
  raise FileNotFoundError("Input directory not found")

if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

We are going to use pretrained vgg model. Remove the output layers(classification task) to get our output features and use those features to fine-tune our model.

In [23]:
inputs_shape = (224, 224, 3) # input shape expected by VGG16
inputs = Input(shape=inputs_shape)
base_model = VGG16(include_top=True, weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
# store image features
image_features = {} # key: imageID without extension, value: feature vector of image

img_dir = os.path.join(INPUT_DIR, 'Images')

# loop through each image
for img_name in tqdm(os.listdir(img_dir)):
  # load image
  img_path = os.path.join(img_dir, img_name)
  img = load_img(img_path, target_size=(224, 224))
  # convert the image pixels to numpy array
  img = img_to_array(img)
  # reshape the image data for the model
  img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
  # preprocess image
  img = preprocess_input(img)
  img_feature = model.predict(img, verbose=0) # get feature vector of the image(value)
  img_id = img_name.split('.')[0] # get the image ID by removing the extension in image(key)
  image_features[img_id] = img_feature


Exception ignored in: <function tqdm.__del__ at 0x00000153586BD1C0>
Traceback (most recent call last):
  File "c:\Users\gauta\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\gauta\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
100%|██████████| 8091/8091 [1:20:44<00:00,  1.67it/s]


In [32]:
print(len(image_features))
image_features['1000268201_693b08cb0e']
print(image_features)

8091
{'1000268201_693b08cb0e': array([[2.5074737, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32), '1001773457_577c3a7d70': array([[0.       , 0.       , 0.4941472, ..., 0.       , 0.       ,
        0.       ]], dtype=float32), '1002674143_1b742ab4b8': array([[1.4937919 , 0.        , 0.53567576, ..., 2.3152387 , 3.741826  ,
        0.        ]], dtype=float32), '1003163366_44323f5815': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32), '1007129816_e794419615': array([[0.        , 0.09227663, 0.        , ..., 0.        , 0.        ,
        0.0652926 ]], dtype=float32), '1007320043_627395c3d8': array([[0.      , 0.      , 0.      , ..., 0.      , 3.339016, 0.      ]],
      dtype=float32), '1009434119_febe49276a': array([[2.0965166, 2.1192532, 3.5619879, ..., 0.642428 , 2.7142   ,
        0.       ]], dtype=float32), '1012212859_01547e3f17': array([[0.        , 0.        , 0.98737216, ..., 0.        , 1.4932494 ,
        0.8612845 ]], dtype=floa

In [33]:
# store image features in pickle
pickle.dump(image_features, open(os.path.join(OUTPUT_DIR, 'img_features.pkl'), 'wb'))

In [65]:
# Load features from pickle file
with open(os.path.join(OUTPUT_DIR, 'img_features.pkl'), 'rb') as file:
    loaded_features = pickle.load(file)

In [66]:
with open(os.path.join(INPUT_DIR,'captions.txt'), 'r') as f:
  next(f)
  captions_doc = f.read()

In [67]:
image_to_caption_mapping = defaultdict(list)

# process lines from captions.txt
for line in tqdm(captions_doc.split('\n')):
  # split by commas(,)
  tokens = line.split(',')
  if len(tokens) < 2:
    continue
  image_id, *captions = tokens # split the tokens into image_id and captions
  image_id = image_id.split('.')[0] # remove the extensions
  caption = " ".join(captions)
  image_to_caption_mapping[image_id].append(caption)

total_captions = sum(len(captions) for captions in image_to_caption_mapping.values())
print(f'Total number of captions: {total_captions}')

100%|██████████| 40456/40456 [00:00<00:00, 1043039.30it/s]

Total number of captions: 40455


In [38]:
# before preprocess
image_to_caption_mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [39]:
# preprocess captions
def preprocess_captions(mapping):
  for key, captions in tqdm(mapping.items()):
    for i in range(len(captions)):
      # one caption at a time
      caption = captions[i]
      # convert to lower case
      caption = caption.lower()
      # delete digits and other special characters
      caption = ''.join(char for char in caption if char.isalnum() or char.isspace())
      # remove extra spaces
      caption = caption.replace('\s+', ' ')
      # add start and end tags
      caption = 'startseq ' + ' '.join([word for word in caption.split() if len(word) > 1]) + ' endseq'
      captions[i] = caption

In [40]:
# after preprocess
preprocess_captions(image_to_caption_mapping)
image_to_caption_mapping['1000268201_693b08cb0e']

100%|██████████| 8091/8091 [00:00<00:00, 31420.44it/s]


['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [41]:
# create a list of all captions
all_captions = [caption for captions in image_to_caption_mapping.values() for caption in captions]

In [42]:
all_captions[:10]

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq',
 'startseq black dog and spotted dog are fighting endseq',
 'startseq black dog and tricolored dog playing with each other on the road endseq',
 'startseq black dog and white dog with brown spots are staring at each other in the street endseq',
 'startseq two dogs of different breeds looking at each other on the road endseq',
 'startseq two dogs on pavement moving toward each other endseq']

In [68]:
# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
print(f'Vocabulary size: {vocab_size}')

Vocabulary size: 8811


In [44]:
# save the tokenizer
with open(os.path.join(OUTPUT_DIR, 'tokenizer.pkl'), 'wb') as tokenizer_file:
  pickle.dump(tokenizer, tokenizer_file)

In [69]:
# load the tokenizer
with open(os.path.join(OUTPUT_DIR, 'tokenizer.pkl'), 'rb') as tokenizer_file:
  tokenizer = pickle.load(tokenizer_file)

In [70]:
vocab_size = len(tokenizer.word_index) + 1
print(f'Vocabulary size: {vocab_size}')

Vocabulary size: 8811


In [71]:
# Calculate maximum caption length
max_caption_length = max(len(tokenizer.texts_to_sequences([caption])[0]) for caption in all_captions)
print(f'Maximum caption length: {max_caption_length}')

Maximum caption length: 34


Creating train test split

In [48]:
# list of image ID's
image_ids = list(image_to_caption_mapping.keys())
# split into train and test sets
train_size = int(0.9 * len(image_ids))
train = image_ids[:train_size]
test = image_ids[train_size:]

In [49]:
print(len(train), len(test))

7281 810


In [50]:
def prepare_data(image_id, image_to_caption_mapping, features, tokenizer, max_caption_length, vocab_size):
    """
    Prepare input-output pairs for a given image and its captions.
    """
    X1, X2, y = [], [], []
    captions = image_to_caption_mapping[image_id]
    for caption in captions:
        # Convert the caption to a sequence of token IDs
        caption_seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(caption_seq)):
            # Input sequence (padded) and one-hot encoded output
            in_seq = pad_sequences([caption_seq[:i]], maxlen=max_caption_length)[0]
            out_seq = to_categorical([caption_seq[i]], num_classes=vocab_size)[0]

            # Append flattened features to match expected model input shape
            X1.append(features[image_id].flatten())  # Ensure shape is (4096,)
            X2.append(in_seq)
            y.append(out_seq)
    return X1, X2, y


def data_generator(image_ids, image_to_caption_mapping, features, tokenizer, max_caption_length, vocab_size, batch_size):
    """
    Generator function to yield batches of input-output pairs.
    """
    X1_buffer, X2_buffer, y_buffer = [], [], []
    while True:
        for image_id in image_ids:
            X1, X2, y = prepare_data(image_id, image_to_caption_mapping, features, tokenizer, max_caption_length, vocab_size)
            X1_buffer.extend(X1)
            X2_buffer.extend(X2)
            y_buffer.extend(y)

            # Yield batches when the buffer reaches batch size
            while len(X1_buffer) >= batch_size:
                yield (
                    (np.array(X1_buffer[:batch_size]), np.array(X2_buffer[:batch_size])),  # Input tuple (X1, X2)
                    np.array(y_buffer[:batch_size]),  # Output y
                )
                # Remove used data from buffers
                X1_buffer = X1_buffer[batch_size:]
                X2_buffer = X2_buffer[batch_size:]
                y_buffer = y_buffer[batch_size:]

In [51]:
from tensorflow.keras.layers import RepeatVector, Bidirectional, Lambda
from tensorflow.keras.layers import Dot, Activation


In [54]:
# Encoder
# image feature layers
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
fe1_projected = RepeatVector(max_caption_length)(fe2)
fe2_projected = Bidirectional(LSTM(256, return_sequences=True))(fe1_projected)

# sequence feature layers
inputs2 = Input(shape=(max_caption_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = Bidirectional(LSTM(256, return_sequences=True))(se2)

# apply attention mechanism
attention = Dot(axes=(2, 2))([fe2_projected, se3])  # compute attention scores
attention_scores = Activation('softmax')(attention)

# Apply attention scores to sequence embeddings
attention_context = Lambda(lambda x: tf.einsum('ijk,ijl->ikl', x[0], x[1]))([attention_scores, se2])

# Sum the attended sequence embeddings along the time axis
context_vector = Lambda(lambda x: tf.reduce_sum(x, axis=1))(attention_context)

# Decoder model
decoder_input = concatenate([context_vector, fe2], axis=-1)
decoder1 = Dense(256, activation='relu')(decoder_input)
outputs = Dense(vocab_size, activation='softmax')(decoder1)

# Create the model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Visualize the model
plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [53]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [56]:
# Set the number of epochs, batch size
epochs = 100
batch_size = 32

# Calculate the steps_per_epoch based on the number of batches in one epoch
steps_per_epoch = ceil(len(train) / batch_size)
validation_steps = ceil(len(test) / batch_size)  # Calculate the steps for validation data

# Loop through the epochs for training
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    # Set up data generators
    train_generator = data_generator(train, image_to_caption_mapping, loaded_features, tokenizer, max_caption_length, vocab_size, batch_size)
    test_generator = data_generator(test, image_to_caption_mapping, loaded_features, tokenizer, max_caption_length, vocab_size, batch_size)

    model.fit(train_generator, epochs=1, steps_per_epoch=steps_per_epoch,
          validation_data=test_generator, validation_steps=validation_steps,
          verbose=1)

Epoch 1/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 59s 261ms/step - loss: 5.2009 - val_loss: 6.3933
Epoch 2/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 79s 344ms/step - loss: 4.7901 - val_loss: 6.9334
Epoch 3/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 69s 303ms/step - loss: 4.7628 - val_loss: 7.2814
Epoch 4/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 80s 351ms/step - loss: 4.7018 - val_loss: 7.6530
Epoch 5/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 91s 398ms/step - loss: 4.6651 - val_loss: 7.1729
Epoch 6/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 119s 520ms/step - loss: 4.6502 - val_loss: 7.3130
Epoch 7/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 106s 463ms/step - loss: 4.6131 - val_loss: 7.7630
Epoch 8/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 92s 402ms/step - loss: 4.4811 - val_loss: 7.5630
Epoch 9/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 78s 343ms/step - loss: 4.3645 - val_loss: 8.8001
Epoch 10/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 132s 579ms/step - loss: 4.2781 - val_loss: 7.5773
Epoch 11/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 118s 516ms/step - loss: 4.1328 - val_loss: 8.2425
Epoc

In [57]:
pip install --upgrade h5py

In [92]:
model.save('model.h5')

In [94]:
# load the model
model = load_model('model.h5')

ValueError: Unknown layer: 'NotEqual'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [75]:
def get_word_from_index(index, tokenizer):
    return next((word for word, idx in tokenizer.word_index.items() if idx == index), None)

In [76]:
def predict_caption(model, image_features, tokenizer, max_caption_length):
    # Initialize the caption sequence
    caption = 'startseq'

    # Generate the caption
    for _ in range(max_caption_length):
        # Convert the current caption to a sequence of token indices
        sequence = tokenizer.texts_to_sequences([caption])[0]
        # Pad the sequence to match the maximum caption length
        sequence = pad_sequences([sequence], maxlen=max_caption_length)
        # Predict the next word's probability distribution
        yhat = model.predict([image_features, sequence], verbose=0)
        # Get the index with the highest probability
        predicted_index = np.argmax(yhat)
        # Convert the index to a word
        predicted_word = get_word_from_index(predicted_index, tokenizer)

        # Append the predicted word to the caption
        caption += " " + predicted_word

        # Stop if the word is None or if the end sequence tag is encountered
        if predicted_word is None or predicted_word == 'endseq':
            break

    return caption

In [86]:
# Initialize lists to store actual and predicted captions
actual_captions_list = []
predicted_captions_list = []

# Loop through the test data
for key in tqdm(test):
    # Get actual captions for the current image
    actual_captions = image_to_caption_mapping[key]
    # Predict the caption for the image using the model
    predicted_caption = predict_caption(model, loaded_features[key], tokenizer, max_caption_length)

    # Split actual captions into words
    actual_captions_words = [caption.split() for caption in actual_captions]
    # Split predicted caption into words
    predicted_caption_words = predicted_caption.split()

    # Append to the lists
    actual_captions_list.append(actual_captions_words)
    predicted_captions_list.append(predicted_caption_words)

# Calculate BLEU score
print("BLEU-1: %f" % corpus_bleu(actual_captions_list, predicted_captions_list, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual_captions_list, predicted_captions_list, weights=(0.5, 0.5, 0, 0)))

  2%|▏         | 13/810 [00:19<19:46,  1.49s/it]


KeyboardInterrupt: 